In [1]:
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers


In [2]:
train_dir = r'C:\Users\Prashant\Desktop\seg_train\seg_train'
test_dir = r'C:\Users\Prashant\Desktop\seg_test\seg_test'
pred_dir = r'C:\Users\Prashant\Desktop\seg_pred'
labels = sorted(os.listdir(train_dir))
labels

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [3]:
datagen = ImageDataGenerator(rescale=1./255,
                           shear_range = 0.2,
                           zoom_range = 0.2,
                           horizontal_flip=True,
                           validation_split=0.2)
train_generator = datagen.flow_from_directory(
                        train_dir,
                        shuffle=False,
                        class_mode="categorical",
                        target_size=(150,150),
                        batch_size = 16,
                        subset='training')
valid_generator = datagen.flow_from_directory(
                        train_dir,
                        shuffle=False,
                        class_mode="categorical",
                        target_size=(150,150),
                        batch_size = 16,
                        subset='validation')

tst_aug = ImageDataGenerator(rescale=1./255)
test_gen = tst_aug.flow_from_directory(
                        test_dir,
                        shuffle=False,
                        class_mode="categorical",
                        target_size=(150,150),
                        batch_size = 16)



Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [4]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import tensorflow as tf


In [5]:
inceptionV3 = InceptionV3(include_top= False, input_shape=(150,150,3))

for layer in inceptionV3.layers:
    layer.trainable = False

In [6]:
last_layer = inceptionV3.get_layer('mixed9')

print('last layer output shape: ', last_layer.output_shape)

last_output = last_layer.output

last layer output shape:  (None, 3, 3, 2048)


In [7]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(units = 1024, activation = tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense  (6, activation = tf.nn.softmax)(x)

model = tf.keras.Model( inceptionV3.input, x)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=1,
                                            verbose=1,
                                            factor=0.25,
                                            min_lr=0.000003)

model.compile(optimizer= 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit_generator(
    train_generator,
    epochs=30,
    validation_data = valid_generator, 
    callbacks=EarlyStopping(monitor='val_loss', mode='min',verbose=1))

Epoch 1/30
702/702 [==============================] - 674s 960ms/step - loss: 0.3553 - accuracy: 0.8742 - val_loss: 0.3454 - val_accuracy: 0.8663
Epoch 2/30
702/702 [==============================] - 672s 957ms/step - loss: 0.2752 - accuracy: 0.8965 - val_loss: 0.3222 - val_accuracy: 0.8852
Epoch 3/30
702/702 [==============================] - 671s 956ms/step - loss: 0.2260 - accuracy: 0.9147 - val_loss: 0.2887 - val_accuracy: 0.8973
Epoch 4/30
702/702 [==============================] - 673s 958ms/step - loss: 0.2065 - accuracy: 0.9219 - val_loss: 0.2845 - val_accuracy: 0.9001
Epoch 5/30
702/702 [==============================] - 670s 954ms/step - loss: 0.1774 - accuracy: 0.9317 - val_loss: 0.2775 - val_accuracy: 0.9062
Epoch 6/30
702/702 [==============================] - 673s 959ms/step - loss: 0.1662 - accuracy: 0.9369 - val_loss: 0.3284 - val_accuracy: 0.9016
Epoch 00006: early stopping


In [10]:
model.evaluate(test_gen)

188/188 [==============================] - 139s 738ms/step - loss: 0.2959 - accuracy: 0.9083


[0.29586097598075867, 0.9083333611488342]

In [12]:
pred_aug = ImageDataGenerator(rescale=1./255)
pred_gen = pred_aug.flow_from_directory(
                        pred_dir,
                        shuffle=False,
                        class_mode="categorical",
                        target_size=(150,150),
                        batch_size = 16)


Found 7301 images belonging to 1 classes.


In [13]:
results = model.predict(pred_gen)
results

array([[9.5239508e-01, 2.4686996e-03, 1.8252898e-02, 1.8092648e-03,
        1.7840877e-04, 2.4895692e-02],
       [7.7899938e-07, 3.3279840e-04, 4.5247074e-02, 9.5439827e-01,
        1.8944007e-05, 2.0044527e-06],
       [2.6867363e-01, 2.1079801e-05, 2.5055872e-04, 1.5427085e-04,
        1.6074962e-05, 7.3088437e-01],
       ...,
       [4.7736335e-09, 1.1178713e-06, 3.9310911e-04, 9.9955004e-01,
        5.5650471e-05, 5.4621103e-09],
       [2.5769602e-13, 1.0000000e+00, 9.6019748e-11, 7.4891927e-17,
        4.2506138e-15, 7.1755086e-13],
       [4.9864808e-05, 3.2543945e-10, 8.3076550e-08, 8.1665757e-11,
        3.8515666e-11, 9.9995005e-01]], dtype=float32)

In [14]:
import keras
model.save('imgintelclass.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: imgintelclass.model\assets


In [15]:
predicted_class_indices=np.argmax(results,axis=1)


In [22]:
predicted_class_indices

array([0, 3, 5, ..., 3, 1, 5], dtype=int64)

In [17]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [18]:
filenames=pred_gen.filenames
res=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

In [19]:
res




,Filename,Predictions
0,seg_pred\10004.jpg,buildings
1,seg_pred\10005.jpg,mountain
2,seg_pred\10012.jpg,street
3,seg_pred\10013.jpg,mountain
4,seg_pred\10017.jpg,mountain
...,...,...
7296,seg_pred\9988.jpg,street
7297,seg_pred\9992.jpg,forest
7298,seg_pred\9993.jpg,mountain
7299,seg_pred\9995.jpg,forest
